In [85]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np

In [86]:
# Dataset class
class VeggieDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images  # Should be in (batch, channels, height, width)
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = np.transpose(self.images[idx], (1, 2, 0))  # Convert (C, H, W) -> (H, W, C)
        image = Image.fromarray((image * 255).astype(np.uint8))  # Convert to PIL Image
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(self.labels[idx], dtype=torch.float32).squeeze()
        return image, label

# Model class
class VeggieVision(nn.Module):
    def __init__(self):
        super(VeggieVision, self).__init__()
        self.base_model = models.mobilenet_v2(pretrained=True)
        self.base_model.features.requires_grad = False  # Freeze base layers
        self.fc1 = nn.Linear(self.base_model.last_channel * 7 * 7, 128)  # Adjust input size here
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.base_model.features(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)  # Linear output for regression
        return x

# Data transformations
train_transform = transforms.Compose([
    transforms.RandomRotation(360),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(224, scale=(0.9, 1.1)),
    transforms.ColorJitter(brightness=0.2),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Helper function to prepare data loaders
def prepare_data_loaders(X, y, validation_split=0.2, batch_size=32):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_split, random_state=42)
    train_dataset = VeggieDataset(X_train, y_train, transform=train_transform)
    val_dataset = VeggieDataset(X_val, y_val, transform=test_transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    return train_loader, val_loader

# Model initialization with scheduler setup
def initialize_model(device, learning_rate=1e-4):
    model = VeggieVision().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Set up a scheduler that reduces the learning rate on plateau of validation loss
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
    
    return model, criterion, optimizer, scheduler


# Training loop
def training_loop(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs=20, print_freq=1):
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        
        # Train the model
        train_loss = train(model, train_loader, criterion, optimizer, device, print_freq=print_freq)
        
        # Validate the model
        val_loss = validate(model, val_loader, criterion, device, print_freq=print_freq)
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")
        
        # Step the scheduler based on the validation loss
        scheduler.step(val_loss)


# Fine-tuning setup
def setup_fine_tuning(model, optimizer, learning_rate=1e-5, n_unfreeze_layers=1):
    fine_tune(model, n_unfreeze_layers)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    return optimizer

# Fine-tuning loop
def fine_tuning_loop(model, train_loader, val_loader, criterion, optimizer, device, fine_tune_epochs=2):
    for epoch in range(fine_tune_epochs):
        train_loss = train(model, train_loader, criterion, optimizer, device)
        val_loss = validate(model, val_loader, criterion, device)
        print(f"Fine-tuning Epoch {epoch+1}/{fine_tune_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

def train(model, dataloader, criterion, optimizer, device, print_freq=1):
    model.train()
    running_loss = 0.0
    total_batches = len(dataloader)

    for batch_idx, (images, labels) in enumerate(dataloader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        
        # Print batch loss every `print_freq` batches
        if (batch_idx + 1) % print_freq == 0:
            print(f"Batch {batch_idx+1}/{total_batches}, Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

def validate(model, dataloader, criterion, device, print_freq=1):
    model.eval()
    running_loss = 0.0
    total_batches = len(dataloader)
    
    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(dataloader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs.squeeze(), labels)
            running_loss += loss.item() * images.size(0)
            
            # Print batch loss every `print_freq` batches
            if (batch_idx + 1) % print_freq == 0:
                print(f"Validation Batch {batch_idx+1}/{total_batches}, Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss


# Fine-tuning function
def fine_tune(model, n_unfreeze_layers=1):
    layers = list(model.base_model.features.children())
    for layer in layers[-n_unfreeze_layers:]:
        for param in layer.parameters():
            param.requires_grad = True

# Evaluation function
def evaluate(model, dataloader, criterion, device):
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs.squeeze(), labels)
            test_loss += loss.item() * images.size(0)
    print(f"Test Loss: {test_loss / len(dataloader.dataset):.4f}")

In [87]:
def main(X, y, batch_size=32, epochs=20, learning_rate=1e-4, validation_split=0.2, fine_tune_epochs=2):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Prepare data loaders
    train_loader, val_loader = prepare_data_loaders(X, y, validation_split, batch_size)
    
    # Initialize model, loss function, optimizer, and scheduler
    model, criterion, optimizer, scheduler = initialize_model(device, learning_rate)
    
    # Training phase with scheduler
    training_loop(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs)

    # Fine-tuning phase
    optimizer = setup_fine_tuning(model, optimizer, learning_rate=1e-5, n_unfreeze_layers=1)
    fine_tuning_loop(model, train_loader, val_loader, criterion, optimizer, device, fine_tune_epochs)

    # Evaluation phase
    evaluate(model, val_loader, criterion, device)


In [88]:
import numpy as np
import torch

# Load the data
X = np.load("../data/processed_data/X.npy")
y = np.load("../data/processed_data/y.npy")

# Convert X from (batch, height, width, channel) to (batch, channel, height, width)
X = np.transpose(X, (0, 3, 1, 2)).astype(np.float32)
y = y.reshape(-1).astype(np.float32)

In [89]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
# Prepare data loaders
train_loader, val_loader = prepare_data_loaders(X, y, 0.2, 32)
    
# Initialize model, loss function, and optimizer
model, criterion, optimizer, scheduler = initialize_model(device, 1e-3)
    

In [90]:
# Training phase
training_loop(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs = 2)

Epoch 1/2
Batch 1/19, Loss: 25047.5566
Batch 2/19, Loss: 8467.7266
Batch 3/19, Loss: 6200.7573
Batch 4/19, Loss: 8364.6162
Batch 5/19, Loss: 4902.1943
Batch 6/19, Loss: 4182.5083
Batch 7/19, Loss: 3084.4709
Batch 8/19, Loss: 225863.6406
Batch 9/19, Loss: 3007.6860
Batch 10/19, Loss: 3475.4595
Batch 11/19, Loss: 7597.6143
Batch 12/19, Loss: 4261.4644
Batch 13/19, Loss: 3177.5859
Batch 14/19, Loss: 3925.0598
Batch 15/19, Loss: 2347.7053
Batch 16/19, Loss: 1572.3467
Batch 17/19, Loss: 3510.8660
Batch 18/19, Loss: 1302.5117
Batch 19/19, Loss: 8025.8115
Validation Batch 1/5, Loss: 1870.8722
Validation Batch 2/5, Loss: 3035.2324
Validation Batch 3/5, Loss: 3262.0908
Validation Batch 4/5, Loss: 3017.9617
Validation Batch 5/5, Loss: 2539.7529
Epoch 1/2, Train Loss: 17660.1766, Validation Loss: 2763.3492
Epoch 2/2
Batch 1/19, Loss: 1331.5555
Batch 2/19, Loss: 2090.3718
Batch 3/19, Loss: 3085.8027
Batch 4/19, Loss: 2452.3711
Batch 5/19, Loss: 1825.3107
Batch 6/19, Loss: 1465.5109
Batch 7/19, Los

In [ ]:
# Fine-tuning phase
optimizer = setup_fine_tuning(model, optimizer, learning_rate=1e-5, n_unfreeze_layers=1)
fine_tuning_loop(model, train_loader, val_loader, criterion, optimizer, device, 2)

In [ ]:
# Evaluation phase
evaluate(model, val_loader, criterion, device)